## 14. 반복형, 반복자, 제너레이터
메모리에 다 들어가지 않는 데이터셋을 검색할 때는 항목들을 느긋하게 가져와야 한다. 즉, 한 번에 하나씩 그리고 필요할 때 가져와야 한다. 이것이 반복자 패턴이 하는 일이다. 파이선은 매크로가 반복자 패턴을 추상화할 수 있게 yield 키워드가 2001년에 파이선 2.2에 추가되었다. yield 키워드는 반복자로 작동하는 제너레이터를 생성할 수 있게 해준다.

먼저 Sentence라는 클래스를 구현하면서 반복형을 알아보자. 이 클래스의 생성자는 텍스트로 구성된 문자열을 받은 후 단어별로 반복할 수 있다. 첫 번째 버전은 시퀀스 프로토콜을 구현하며 반복할 것이다. 앞에서도 설명한 것처럼 모든 시퀀스는 반복할 수 있기 때문이다. 그러나 여기서는 왜 그렇게 되는지 알아볼 것이다. 

In [14]:
""" [예제 14-1] 인덱스를 사용해서 텍스트에서 단어를 추출하는 Sectence 클래스다. """
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text) # re.findall()은 정규 표현식에 매칭되는 문자열의 리스트를 반환한다.
        
    def __getitem__(self, index):          
        return self.words[index] 
    
    def __len__(self): # 완전히 시퀀스 프로토콜에 따르려면 구현해야 하지만, 반복형 객체에 이 메서드가 필요한 것은 아니다.
        return len(self.words)
    
    def __repr__(self):
        return 'Sentence ({})'.format(reprlib.repr(self.text)) # 유틸리티 함수로서, 매우 큰 데이터 구조체를 표현하는 문자열을 축약해서 생성한다.
                                                               # 기본적으로 생상헐 문자열을 30자로 제한한다.

In [15]:
s = Sentence("The time has come, the Walrus said")
s # __repr__()를 이용해서 출력한 메시지는 실제로 reprlib.repr() 의 결과를 반영한다.

Sentence ('The time has...e Walrus said')

In [16]:
for word in s:
    print(word)

The
time
has
come
the
Walrus
said


In [17]:
list(s) # 반복할 수 있으므로 Sentence 객체는 리스트 혹은 다른 반복형을 생성하기 위한 입력으로 사용할 수 있다.

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

In [18]:
s[0], s[5], s[-1] # 인덱스를 이용해서 단어를 가져올 수 있다.

('The', 'Walrus', 'said')

#### 14.1.1 Sequence가 반복 가능한 이유 : iter( ) 함수

<ol>
    <li> 객체가 __iter__( ) 메서드를 구현하는지 확인하고, 이 메서드를 호출해서 반복자를 가져온다. </li>
    <li> __iter__( ) 메서드가 구현되어 있지 않지만 __getitem__( ) 이 구현되어 있다면, 파이썬은 인덱스 0에서 시작해서 항목을 순서대로 가져오는 반복자를 생성한다. 하위 호환성을 위한 것으로 언젠가는 사라질 것이다. </li>
    <li> 이 과정이 모두 실패하면 파이썬은 'TypeError: 'C' object is not iterable' 이라는 메시지와 함께 TypeError가 발생한다. 여기서 'C'는 대상 객체의 클래스다. </li>
</ol>
그렇기 때문에 모든 파이썬 시퀀스는 반복할 수 있다. 시퀀스가 __getitem__( ) 구현하고 있기 때문이다. 사실 표준 시퀀스는 __iter__( )을 구현하고 있으므로 여러분이 정의한 시퀀스도 이 메서드를 구현해야 한다. 11.2절 '파이썬은 시퀀스를 찾아낸다'에서 설명한 것처럼 __iter__( ) 특별 메서드를 구현하고 객체뿐만 아니라 0에서 시작하는 정수형 키를 받는 __getitem__( ) 메서드를 구현하는 객체도 반복형으로 간주하는 것은 덕 타이핑의 극단적인 형태이다. <a href="https://dgkim5360.github.io/blog/python/2017/07/duck-typing-vs-goose-typing-pythonic-interfaces/">구스 타이핑 기법</a>을 사용하면 융통성이 떨어져서 __iter__( ) 특별 메서드를 구현하는 객체만 반복형이라고 간주한다. 

In [19]:
class Foo:
    def __iter__(self):
        pass
    
from collections import abc
issubclass(Foo, abc.Iterable)

True

In [20]:
f = Foo()
isinstance(f, abc.Iterable)

True

In [21]:
isinstance(s, abc.Iterable) # 반복형으로 사용하고 있지만 __iter__() 를 구현하지 않았으므로 테스트를 통과하지 못한다.

# 현재 객체가 반복형인 확인하는 가장 정확한 방법은 iter(x)를 호출하고 만일에 발생할 수 있는 TypeError를 처리하는 것이다. 이 방법이 isinstance(x, abc.Iterable)을 사용하는 방법보다 더 정확하다.

False

### 14.2 반복형과 반복자
<ul>
    <li> 반복형: iter( ) 내장 함수가 반복자를 가져올 수 있는 모든 객체와 반복자를 반환하는 __iter__( ) 메서드가 있는 객체는 반복형이다. 0에서 시작하는 인덱스를 받는 __getitem__( ) 메서드가 있는 시퀀스도 마찬가지다. 반복자에 대한 표준 인터페이스는 다음과 같은 메서드 두 개를 정의한다.</li>
    <ul>
        <li> __next__( ) 다음에 사용할 항목을 반환한다. 더 이상 항목이 남아 있지 않으면 StopIteration을 발생시킨다. </li>
        <li> __iter__( ) self를 반환한다. 그러면 for로프 등 반복형이 필요한 곳에 반복자를 사용할 수 있게 해준다.</li> 
    </ul>
    <li> 반복자: 다음 항목을 반환하거나, 다음 항목이 없을 때 StopIteration 예외를 발생시키도록 인수를 받지 않는 __next__( ) 메서드를 가진 객체이다. 파이썬 반복자는 __iter__( ) 메서드도 구현하므로 반복형이기도 한다. </li>
</ul>

In [22]:
s = 'ABC' # 'ABC'는 반복형이다. 반복자는 내부 어디에 숨어 있다.
for char in s:
    print(char)

A
B
C


In [23]:
""" while 문으로 for 흉내내기 """
s = 'ABC'
it = iter(s) # 반복자 생성
while True:
    try:
        print(next(it))  # next 호출을 통해 다음 항목을 가져온다.
    except StopIteration:# 더 이상 항목이 없으면 반복자가 StopIteration 예외를 발생시킨다.
        del it           # it에 대한 참조를 해제해서 반복자 객체를 제거한다. 
        break

A
B
C


[그림 14-1] Iterable과 Iterator ABC, 이탤릭체는 추상 메서드를 나타낸다. Iterable의 구상 서브클래스의 \_\_Iter\_\_( ) 메서드는 Iterator 객체를 생성하고 반환해야 하며, Iterator의 구상 서브 클래스는 \_\_next\_\_( ) 메서드를 구현해야 한다. Iterator.\_\_Iter\_\_( ) 메서드는 self를 반환해야 한다.

<img src="Figure14-1.png">

In [24]:
""" [예제14-3] abc.Iterator 클래스의 일부 """
# class Iterator(Iterable):
    
#     __slots__ = () # 9장 참조
    
#     @abstractmethod
#     def __next__(self):
#         ''' 반복자에서 다음 항목을 반환한다. 항목이 소진되면 StopIteration 예외를 발생시킨다. '''
#         raise StopIteration

#     def __iter__(self):
#         return self
    
#     @classmethod
#     def __subclasshook__(cls, C):
#         if cls is Iterator:
#             if(any("__next__" in B.__dict__ for B in C.__mro__) and 
#                any("__iter__" in B.__dict__ for B in C.__mro__)):
#                return True
#         return NotImplemented

' [예제14-3] abc.Iterator 클래스의 일부 '

In [25]:
s3 = Sentence('Pig and Pepper')
it = iter(s3)
it # doctest: +ELLIPSIS

In [26]:
print(next(it))
print(next(it))
print(next(it))
print(next(it)) # 더 이상 단어가 없으므로 반복자가 StopIteration 예외를 발생시킨다.

Pig
and
Pepper


StopIteration: 

In [27]:
list(it) # 소진된 후에는 반복자가 사라진다.

[]

In [28]:
list(iter(s3)) # 다시 반복하려면 생성자를 새로 만들어야 한다.

['Pig', 'and', 'Pepper']

### 14.3 Sentence 버전 #2: 고전적인 반복자
Sentence 클래스의 다음 버전은 디자인 패텅의 청사진에 따라 고전적인 반복자 패턴에 맞춰 구현한다. 이 코드는 파이선의 관용적인 방법은 아니다. 그렇지만 반복형 컬렉션과 반복자 객체 간의 관계를 명확히 정의하는 데 도움이 된다.

[예제 14-4]를 보면 Sentence 클래스가 반복형이다. \_\_iter\_\_ ( ) 특별 메서드를 구현하고 있고, 이 메서드가 SentenceIterator를 반환하기 때문이다. 이 방식이 디자인 패턴에서 설명하고 있는 반복자 디자인 패턴이다. 여기서는 반복형과 반복자의 차이 및 이 둘이 어떻게 연관되는지 명확히 보여주기 위해 다음과 같이 구현한다. 

In [29]:
""" [예제 14-4] 반복자 패턴을 이용한 Sentence 구현 """
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence: # 반복형
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __repr__(self):
        return 'Sentence({ })'.format(reprlib.repr(self.text))
    
    def __iter__(self): # __getitem__() 메서드가 정의되어 있지 않다. __iter__() 를 구현함으로써 반복형이 된다는 것을 알 수 있다.
        return SentenceIterator(self.words) # __iter__()가 반복자 객체를 생성해서 반환함으로써 반복형 프로토콜을 완전히 구현한다.
                                            
    
class SentenceIterator: # 반복자
    def __init__(self, words):
        self.words = words
        self.index = 0
        
    def __next__(self):
        try:
            word = self.word[self.index]
        except IndexError:
            raise StopIteration()
        self.index += 1
        return word
    
    def __iter__(self): # 사실 구현할 필요가 없지만 반복자는 __next__()와 __iter__() 메서드를 모두 구현해야 하며
        return self     # 둘 다 구현하면 issubclass(SentenceIterator, abc.Iterator) 테스트를 통과할 수 있다.  

#### 14.3.1 Sentence를 반복자로 만들기: 좋지 않은 생각

반복형고 반복자를 만드는 데 있어서 흔히 발생하는 오류는 둘을 혼동하기 때문이다. 간단히 정리하면, 반복형은 호출될 때마다 반복자를 새로 생성하는 \_\_iter\_\_( ) 메서드를 가지고 있다. 반복자는 개별 항목을 반환하는 \_\_next\_\_( ) 메서드와 self를 반환하는 \_\_iter\_\_( ) 메서드를 가지고 있다. 따라서 반복자는 반복형이지만 반복형은 반복자가 아니다.

반복자 패턴은 다음과 같은 용도에 사용한다.
<li> 집합 객체의 내부 표현을 노출시키지 않고 내용에 접근하는 경우 </li>
<li> 집합 객체의 다중 반복을 지원하는 경우 </li>
<li> 다양한 집합 구조체를 반복하기 위한 통일된 인터페이스를 제공하는 경우 </li>

'다중 반복을 지원'하려면 동일한 반복형 객체로부터 여러 독립적인 반복자를 가질 수 있어야 하며, 각 반복자는 고유한 내부 상태를 유지해야 한다. 따라서 이 패턴을 제대로 구현하려면 iter(my_iterable)을 호출할 때마다 독립적인 반복자가 새로 만들어져야 한다. 그렇기 때문에 이 예제의 SentenceIterator 클래스가 필요한 것이다. 

반복형은 자기 자신을 반복하는 반복자로 작동하면 안된다. 즉 \_\_iter\_\_( )를 구현하되, \_\_next\_\_( )는 구현하면 안된다. 한편 편의를 위해 반복자는 반복형이 되어야 한다. 이때 반복자의 \_\_iter\_\_( )는 self를 반환해야 한다.

### 14.4 Sentence 버전 #3: 제너레이터 함수
동일한 기능을 파이썬스럽게 구현하려면 SequenceIterator 클래스 대신 제너레이터 함수를 사용한다.

In [30]:
""" [예제 14-5] 제너레이터 함수를 사용해서 구현한 Sentence """

import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence: # 반복형
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __repr__(self):
        return 'Sentence({ })'.format(reprlib.repr(self.text))
    
    def __iter__(self):
        for word in self.words:
            yield word # 제너레이터 객체로서 자동으로 생성된다.
        return         # 함수가 끝에 도달하면 자동으로 반환하므로 return은 필요없다. 그리고 제너레이터 함수는 StopIteration도 발생시키지 않는다. 값을 모두 생산한 후 그냥 빠져나간다.

#### 14.4.1 제너레이터 함수의 작동 방식
본체 안에 Yield 키워드를 가진 함수는 모두 제너레이터 함수다. 제너레이터 함수는 호출되면 제너레이터 객체를 반환한다. 일반 함수와 제너레이터 함수는 본체 안 어디에선가 yield 키워드를 사용한다는 차이 뿐이다. 다음은 제너레이터의 작동을 간단하게 보여준다.

In [31]:
def gen_123(): # yield 키워드를 포함하고 있는 함수는 모두 제너레이터 함수이다.
    yield 1
    yield 2
    yield 3

In [32]:
gen_123 # doctest: +ELLIPSIS
        # 함수 객체임을 알 수 있다.

<function __main__.gen_123()>

In [33]:
gen_123() # 그러나 호출하면 제너레이터 객체를 반환한다.

<generator object gen_123 at 0x7f43e4e3b678>

In [34]:
for i in gen_123(): # 제너레이터는 yield에 전달된 표현식의 값을 생성하는 반복자다.
    print(i)

1
2
3


In [35]:
g = gen_123() # g가 반복자이기도 하므로 nex(g)로 호출하면 yield가 생성한 다음 항목을 가져온다.
print(next(g))
print(next(g))
print(next(g))
print(next(g))

1
2
3


StopIteration: 

※ 참고사항 : 제너레이터는 값을 생성한다고 이야기했다. 그러나 제너레이터가 값을 '반환'한다고 하면 혼란스럽다. 함수는 값을 반환한다. 제너레이터 함수를 호출하면 제너레이터 객체가 반환된다. 제너레이터 객체는 값을 생성한다. 제너레이터 객체는 일반적인 방식으로 값을 '반환'하지 않는다. 제너레이터 함수 안에 있는 return 문은 제너레이터 객체가 StopIteration 예외를 발생하게 만든다.

In [36]:
""" [예제 14-6] 실행할 때 메시지를 출력하는 제너레이러 함수 """
def gen_AB():
    print('start') # 처음 next()를 암묵적으로 호출하면 'start'를 출력하고, 첫 번째 yield문에서 멈춰 값 'A'를 생성한다.
    yield 'A'
    print('continue') # 두 번째 next()를 암묵적으로 호출하면 'continue'를 출력하고, 두 번째 yield문에서 멈춰 값 'B'를 생성한다.
    yield 'B'
    print('end') # 세 번째 next()를 암묵적으로 호출하면 'end'를 출력하고, 함수 본체의 끝까지 실행되어 제어레이터 객체가 StopIteration을 발생시킨다.
                 # 이 예외를 잡은 후 깔끔하게 루프를 종료한다.

In [37]:
for c in gen_AB():   # for 루프는 g = iter(get_AB())와 대응한 문장을 실행해서 제너레이터 객체를 가져오고 매번 반복할 때 마다 next(g)를 호출한다.
    print('--->', c) # 루프 안에서는 '-->' 문자열 뒤에 next(g)가 반환한 값을 출력한다. 그러나 이 출력은 제너레이터 함수 안의 print()문 다음에 나온다.

start
---> A
continue
---> B
end


### 14.5 Sentence 버전 #4: 느긋한 구현
우리가 지금까지 구현한 Sentence 버전은 느긋한 버전이 아니다. \_\_init\_\_( )에서 택스트안에 있는 단어들의 리스트를 조급하게<sup>eager evaluation</sup> 생성해서 self.words 속성에 바인당하기 때문이다. 그러므로 전체 텍스트를 처리해야 하며, 리스트는 거의 텍스트와 맞먹는 양의 메모리를 소비한다. 

re.finditer( ) 함수는 re.findall( )의 느긋한 버전으로, 리스트 대신 필요에 따라 re.Match Object 객체를 생성하는 제너레이터를 반환한다. 매칭되는 항목이 많으면 re.finditer( )가 메모리를 많이 절약해준다. re.finditer( )를 사용하는 세 번째 버전의 Sentence 클래스는 느긋하게 처리한다. 필요할 때만 다음 단어를 생성하기 때문이다. 

In [38]:
""" [예제 14-7] 제너레이터 함수를 호출하는 생성자 함수를 이용해서 구현한 Sentence """

import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text 
        # self.words = RE_WORD.findall(text) # 단어 리스트를 미리 만들지 않는다.
        
    def __repr__(self):
        return 'Sentence({})'.format(reprlib.repr(self.text))
    
    def __iter__(self):
        for match in RE_WORD.finditer(self.text): # finditer()는 self.text에서 RE_WORD에 대응되는 단어들의 반복자인 MatchObject 객체를 생성한다.
            yield match.group()

### 14.6 Sentence 버전 #5: 제너레이터 표현식
제너레이터 함수도 멋진 방법이지만 제너레이터 표현식을 사용하면 코드를 훨씩 더 짧게 만들 수 있다.

In [39]:
""" [예제 14-8] 지능형 리스트와 제너레이터 표현식에 사용된 gen_AB() """
def gen_AB():
    print('start') 
    yield 'A'
    print('continue') 
    yield 'B'
    print('end')
    
res1 = [x*3 for x in gen_AB( )] # 조급한 반복

for i in res1:
    print('-->', i)

start
continue
end
--> AAA
--> BBB


In [40]:
res2 = (x*3 for x in gen_AB( )) # 느긋한 반복
res2 # res2는 제너레이터 객체이다.

<generator object <genexpr> at 0x7f43e4e3b4c0>

In [41]:
for i in res2: # for 루프가 반복될 때마다 암묵적으로 next(res2)를 호출해서 gen_AB()안에서 다음 yield로 진행하게 만든다.
    print('-->', i)

start
--> AAA
continue
--> BBB
end


In [42]:
""" [예제 14-7] 제너레이터 함수를 호출하는 생성자 함수를 이용해서 구현한 Sentence """
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text 
        # self.words = RE_WORD.findall(text) # 단어 리스트를 미리 만들지 않는다.
        
    def __repr__(self):
        return 'Sentence({})'.format(reprlib.repr(self.text))
    
    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))

### 14.7 제너레이터 표현식: 언제 사용하나?
[예제 10-16] 에서 Vector 클래스를 구현할 때 제너레이터 표현식을 여러 번 사용했다. \_\_eq\_\_( ), \_\_hash\_\_( ), \_\_abs\_\_( ), \_\_angle\_\_( ), \_\_angles\_\_( ), \_\_format\_\_( ), \_\_add\_\_( ), \_\_mul\_\_( ) 메서드가 각각 제너레이터 표현식을 사용한다. 지능형 리스트를 사용해도 제대로 작동하겠지만 그러만 중간의 리스트 값을 저장하기 위해 메모리를 더 많이 사용한다.

제너레이터 함수와 표현식 두 가지 방식 중에서 어떤 것을 선택할 것인지에 대한 필자의 규칙은 간단하다. 제너레이터 표현식이 여러 줄에 걸쳐 있을 때는 가독성을 위해 제너레이터 함수를 사용한다. 게다가 제너레이터 함수는 이름을 가지고 있으므로 재사용할 수도 있다. 

※ 참고사항 : 제너레이터 표현식을 함수나 생성자에 단일 인수로 전달할 때는 함수를 호출하는 괄호 안에서 제너레이터 표현식을 괄호로 에워쌀 필요가 없다. 그러나 제너레이터 표현식 다음에 함수인수가 더 있다면, 구문 에러를 피하기 위해 제너레이터 표현식을 괄호로 에워싸야한다.
```
def __mul__(self,scalar):
    if isinstance(scalar, numbers.Real): # 괄호를 생략함
        return Vector(n * scalar for n in self)
    else:
        return NotImplemented
```

### 14.8 또 다른 예제: 등차수열 제너레이터

전통적인 반복자 패턴은 모두 데이터 구조체를 뒤져서 항목들을 나열하기 위한 것이다. 그러나 수열에서 다음 항목을 가져오는 메서드에 기반한 표준 인터페이스는 컬레션에서 항목을 가져오는 대신 힐행 도중에 항목을 생성하는 경우에도 유용하게 사용할 수 있다.

In [43]:
""" [예제 14-11] ArithmeticProgression 사용 예 """

class ArithmeticProgression:
    def __init__(self, begin, step, end=None):
        self.begin = begin
        self.step = step
        self.end = end
        
    def __iter__(self):
        result = type(self.begin + self.step)(self.begin) # 여기서는 self.begin과 같은 값이 결과가 되지만, 이후에 더할 값에 맞춰 자료형을 강제로 변환한다.
        forever = self.end is None
        index = 0
        while forever or result < self.end:
            yield result
            index += 1
            result = self.begin + self.step * index

In [44]:
""" [예제 14-12] ariprog_gen() 제너테이터 함수 """

def ariprog_gen(begin, step, end=None):
    result = type(begin + step)(begin)
    forever = end is None
    index = 0
    while forever or result < self.end:
        yield result
        index += 1
        result = begin + step * index

In [45]:
""" [예제 14-10] ArithmeticProgression 사용 예 """
ap = ArithmeticProgression(0, 1, 3)
list(ap)

[0, 1, 2]

In [46]:
ap = ArithmeticProgression(0, .5, 3)
list(ap)

[0.0, 0.5, 1.0, 1.5, 2.0, 2.5]

In [47]:
ap = ArithmeticProgression(0, 1/3, 3)
list(ap)

[0.0,
 0.3333333333333333,
 0.6666666666666666,
 1.0,
 1.3333333333333333,
 1.6666666666666665,
 2.0,
 2.333333333333333,
 2.6666666666666665]

In [48]:
from fractions import Fraction
ap = ArithmeticProgression(0, Fraction(1, 3), 1)
list(ap)

[Fraction(0, 1), Fraction(1, 3), Fraction(2, 3)]

In [50]:
from decimal import Decimal
ap = ArithmeticProgression(0, Decimal('.1'), .3)
list(ap)

[Decimal('0'), Decimal('0.1'), Decimal('0.2')]

#### 14.8.1 itertools를 이용한 등차수열
파이썬 3.4 기준으로 itertools 모듈에는 다양하고 재미있게 조합할 수 있는 제너레이터 함수가 19개 들어있다. itertools.count() 함수는 숫자를 생성하는 제너레이터를 반환한다. 인수를 지정하지 않으면 0에서 시작하는 수열을 생성하지만, start와 stop 인수를 지정하면 앞에서 구현한 ariprog_gen( )함수와 아주 비슷한 결과를 낼 수 있다. 그러나 끝을 설정할 수는 없다. itertools.takewhile( )은 이러한 단점을 보완한다. 이 함수는 다른 제너레이터를 소비하면서 주어진 조건식이 False가 되면 중단한다.

In [51]:
import itertools
gen = itertools.count(1, .5)
next(gen), next(gen), next(gen)

(1, 1.5, 2.0)

In [52]:
gen = itertools.takewhile(lambda n: n < 3, itertools.count(1, 0.5))
list(gen)

[1, 1.5, 2.0, 2.5]

In [53]:
""" [예제 14-13] 이전 artprog_gen() 함수와 동일하게 작동 """
import itertools

def aritprog_gen(begin, step, end=None):
    first = type (begin + step)(begin)
    ap_gen = itertools.count(first, step)
    if end is not None:
        ap_gen = itertools.takewhile(lambda n: n < end, ap_gen)
    return ap_gen  # yield문이 없으므로 제너레이터 함수가 아니지만 다른 제너레이터 함수와 마찬가지로 일종의 제너레이터 팩토리처럼 동작한다.

In [54]:
from fractions import Fraction
ap = aritprog_gen(0, 1, 3)
list(ap)

[0, 1, 2]

### 14.9 표준 라이브러리의 제너레이터 함수
예제 14-13에서 설명하고자 하는 것은 제너레이터를 구현할 때 표준 라이브러리에서 어떤 것이 제공되고 있는지 확인하라는 것이다. 이 절에서 설명하는 함수를 모두 알고 있을 수도 있지만, 일부 함수는 잘 사용되지 않는다. 따라서 이미 제공되는 있는 함수를 다시 한 번 쭉 둘러보는 정도로 읽어보는 것도 나쁘지 않다.

In [55]:
""" [예제 14-14] 필터링 제너레이터 함수 예 """
import itertools

def vowel(c): # 모음 문자에 True를 반환
    return c.lower() in 'aeiou'

# filter(predicate, it) predicate를 it의 각 항목에 적용해서 predicate(it)가 참된 값이면 각 항목을 생성한다.
# predicate가 None 이면 참된 항목을 모두 생성한다. 
print(list(filter(vowel, 'Aardvarke'))) 

# filter()의 반대 논리를 적용한다. 
print(list(itertools.filterfalse(vowel, 'Aardvarke')))

# (?) takewhile(predicate, it) predicate가 참된 값으로 계산되는 동안 모은 항목을 생성하고 추가 검사없이 멈춘다.
print(list(itertools.takewhile(vowel, 'Aardvarke')))

# 두 개의 반복형을 병렬로 소비한다. selector_it의 해당 항목이 참된 값일 때마다 it에서 항목을 생성한다.
print(list(itertools.compress('Aardvarke', (1,0,1,1,0,1))))

# islice(it.stop)이나 islice(it, start, stop, step=1) 
# s[:stop]이나 s[start:stop:step] 과 비슷하게 반복할 수 있는 모든 객체에 느긋하게 연산을 ㅈ거용해서 it의 슬라이스 항목을 생성한다.
print(list(itertools.islice('Aardvarke', 4)))
print(list(itertools.islice('Aardvarke', 4, 7)))
print(list(itertools.islice('Aardvarke', 1, 7, 2)))

['A', 'a', 'a', 'e']
['r', 'd', 'v', 'r', 'k']
['A', 'a']
['A', 'r', 'd', 'a']
['A', 'a', 'r', 'd']
['v', 'a', 'r']
['a', 'd', 'a']


In [56]:
""" [예제 14-15] 매핑 제너레이터: itertools.accumulate() """
sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]

import itertools
print(list(itertools.accumulate(sample)))
print(list(itertools.accumulate(sample, min)))
print(list(itertools.accumulate(sample, max)))

import operator
print(list(itertools.accumulate(sample, operator.mul)))
print(list(itertools.accumulate(range(1, 11), operator.mul)))

[5, 9, 11, 19, 26, 32, 35, 35, 44, 45]
[5, 4, 2, 2, 2, 2, 2, 0, 0, 0]
[5, 5, 5, 8, 8, 8, 8, 8, 9, 9]
[5, 20, 40, 320, 2240, 13440, 40320, 0, 0, 0]
[1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]


In [57]:
""" [예제 14-16] 매핑 제너레이터 함수 예 """

print(list(enumerate('albatroz', 1)))

import operator
print(list(map(operator.mul, range(11), range(11))))
print(list(map(operator.mul, range(11), (1,2,3))))
print(list(map(lambda a, b: (a, b), range(11), [2, 4, 8]))) # 두 반복형에서 나란히 숫자를 가져온다. 가장 짧은 반복형이 끝나자마자 생성을 중단한다.

import itertools
# starmap(func, it) it의 각 항목에 func를 적용해서 결과를 생성한다. 입력된 it는 iit 항목을 생성하고 func(*iit) 형태로 호출된다.
print(list(itertools.starmap(operator.mul, enumerate('albatroz', 1)))) # 1부터 시작해서 단어 안의 각 글자를 인덱스만큼 반복한다.

sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]
print(list(itertools.starmap(lambda a, b: b/a, enumerate(itertools.accumulate(sample), 1)))) # 이동 평균을 구한다.

[(1, 'a'), (2, 'l'), (3, 'b'), (4, 'a'), (5, 't'), (6, 'r'), (7, 'o'), (8, 'z')]
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
[0, 2, 6]
[(0, 2), (1, 4), (2, 8)]
['a', 'll', 'bbb', 'aaaa', 'ttttt', 'rrrrrr', 'ooooooo', 'zzzzzzzz']
[5.0, 4.5, 3.6666666666666665, 4.75, 5.2, 5.333333333333333, 5.0, 4.375, 4.888888888888889, 4.5]


In [58]:
""" [예제 14-17] 병합 생성자 함수 예 """
print(list(itertools.chain('ABC', range(2))))
print(list(itertools.chain(enumerate('ABC'))))
print(list(itertools.chain.from_iterable(enumerate('ABC')))) # 반복형에서 항목을 하나씩 가져와서, 각 항목이 반복형이면 시퀀스 안에 연결한다.

print(list(zip('ABC', range(5)))) # 각 it의 항목을 병렬로 소비해서 N-튜플을 생성한다. 짧은 it이 모두 소모되면 조용히 중단한다. 
print(list(itertools.zip_longest('ABC', range(5)))) # 짧은 it이 모두 소모되면 None로 채운다
print(list(itertools.zip_longest('ABC', range(5), fillvalue='?')))  # None을 fillvalue로 대신한다.

['A', 'B', 'C', 0, 1]
[(0, 'A'), (1, 'B'), (2, 'C')]
[0, 'A', 1, 'B', 2, 'C']
[('A', 0), ('B', 1), ('C', 2)]
[('A', 0), ('B', 1), ('C', 2), (None, 3), (None, 4)]
[('A', 0), ('B', 1), ('C', 2), ('?', 3), ('?', 4)]


In [59]:
""" [예제 14-18] itertools.product() 제너레이터 함수 예 """

# 다양한 형태로 데카르트 곱을 느긋하게 계산한다.
print(list(itertools.product('ABC', range(2))))
suits = 'spades hearts diamonds clubs'.split()
print(list(itertools.product('AK', suits)))
print(list(itertools.product('ABC')))
print(list(itertools.product('ABC', repeat=2)))
print(list(itertools.product(range(2), repeat=3)))

rows = itertools.product('AB', range(2), repeat=2) # repeat=N 키워드 인수는 입력된 각 반복형을 N번 소비하라고 product()에 알려준다.
for row in rows: print(row)

[('A', 0), ('A', 1), ('B', 0), ('B', 1), ('C', 0), ('C', 1)]
[('A', 'spades'), ('A', 'hearts'), ('A', 'diamonds'), ('A', 'clubs'), ('K', 'spades'), ('K', 'hearts'), ('K', 'diamonds'), ('K', 'clubs')]
[('A',), ('B',), ('C',)]
[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'A'), ('B', 'B'), ('B', 'C'), ('C', 'A'), ('C', 'B'), ('C', 'C')]
[(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)]
('A', 0, 'A', 0)
('A', 0, 'A', 1)
('A', 0, 'B', 0)
('A', 0, 'B', 1)
('A', 1, 'A', 0)
('A', 1, 'A', 1)
('A', 1, 'B', 0)
('A', 1, 'B', 1)
('B', 0, 'A', 0)
('B', 0, 'A', 1)
('B', 0, 'B', 0)
('B', 0, 'B', 1)
('B', 1, 'A', 0)
('B', 1, 'A', 1)
('B', 1, 'B', 0)
('B', 1, 'B', 1)


In [60]:
""" [예제 14-19] 입력된 항목 하나를 여러 개로 확장하는 제너레이터 함수 """
""" count(), cycle(), repeat() """

ct = itertools.count()
print(next(ct), next(ct), next(ct), next(ct)) # ct는 끝이 없기 때문에 list로 생상할 수 없음
print(list(itertools.islice(itertools.count(1, .3), 3)))

cy = itertools.cycle('ABC')
next(cy)
print(list(itertools.islice(cy, 7))) # ABCABCABCABC 순으로 순환함, next 호출 이후 7이므로 'B'로 끝남

rp = itertools.repeat(7)
print(next(rp), next(rp))           # 7을 next 횟수만큼 반복
print(list(itertools.repeat(8, 4))) # 8을 4번 반복
print(list(map(operator.mul, range(11), itertools.repeat(5)))) # 모두 5를 곱한다.

0 1 2 3
[1, 1.3, 1.6]
['B', 'C', 'A', 'B', 'C', 'A', 'B']
7 7
[8, 8, 8, 8]
[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]


In [61]:
""" [예제 14-20] 하나의 입력 항목마다 값을 여러 개 생성하는 순열 조합 제노레이터 함수 """
print(list(itertools.combinations('ABC', 2)))   # 길이가 2인 조합을 모두 생성한다.
print(list(itertools.combinations_with_replacement('ABC', 2))) # 길이가 2인 조합을 모두 생성한고 항목을 반복할 수 있다.
print(list(itertools.permutations('ABC', 2)))   # 길이가 2인 순열을 모두 구한다.
print(list(itertools.product('ABC', repeat=2))) # 데카르트 곱을 구한다.

[('A', 'B'), ('A', 'C'), ('B', 'C')]
[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'B'), ('B', 'C'), ('C', 'C')]
[('A', 'B'), ('A', 'C'), ('B', 'A'), ('B', 'C'), ('C', 'A'), ('C', 'B')]
[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'A'), ('B', 'B'), ('B', 'C'), ('C', 'A'), ('C', 'B'), ('C', 'C')]


In [62]:
""" [예제 14-21] 재배치 제너레이터 함수 """
""" itertools.groupby()와 내장 함수 reversed() 사용 예 """

print('{!r}'.format(list(itertools.groupby('LLLLAAGGG'))))

print('\n')
for char, group in itertools.groupby('LLLLAAGGG'):
    print(char, '->', list(group))
    
print('\n')
animals = ['duck', 'eagle', 'rat', 'giraffe', 'bear', 'bat', 'dolphin', 'shark', 'lion']
for length, group in itertools.groupby(animals, len):
    print(length, '->', list(group))

print('\n')    
for length, group in itertools.groupby(reversed(animals), len):
    print(length, '->', list(group))

[('L', <itertools._grouper object at 0x7f43e4dfe470>), ('A', <itertools._grouper object at 0x7f43e4dfe4a8>), ('G', <itertools._grouper object at 0x7f43e4dfe4e0>)]


L -> ['L', 'L', 'L', 'L']
A -> ['A', 'A']
G -> ['G', 'G', 'G']


4 -> ['duck']
5 -> ['eagle']
3 -> ['rat']
7 -> ['giraffe']
4 -> ['bear']
3 -> ['bat']
7 -> ['dolphin']
5 -> ['shark']
4 -> ['lion']


4 -> ['lion']
5 -> ['shark']
7 -> ['dolphin']
3 -> ['bat']
4 -> ['bear']
7 -> ['giraffe']
3 -> ['rat']
5 -> ['eagle']
4 -> ['duck']


In [63]:
""" [예제 14-22] itertools.tee()는 여러 개의 제너레이터를 생성하며, 각 제너레이터는 입력된 내용을 그대로 생성한다. """

# 입력된 하나의 반복형에 대한 여러 제너레이터를 생성하고, 각 제너레이터는 독립적으로 항목들을 반환한다.
list(itertools.tee('ABC'))

g1, g2 = itertools.tee('ABC')
print(next(g1), next(g2), next(g2))

print(list(g1))
print(list(g2))

list(zip(*itertools.tee('ABC')))

A A B
['B', 'C']
['C']


[('A', 'A'), ('B', 'B'), ('C', 'C')]

### 14.10 파이썬 3.3의 새로운 구문: yield from
다른 제너레이터에서 생성된 값을 상위 제너레이터 함수가 생성해야할 때는 전통적으로 중첩된 for루프를 사용했다. 그러나 PEP 380 - 하위제너레이터에 위임하기 위한 구문(Syntax for Delegating to a subgenerator)에서는 새로운 구문을 소개했다. 

In [64]:
""" 전통적인 방식 """
def chain(*iterables):
    for it in iterables:
        for i in it:
            yield i

s = 'ABC'
t = tuple(range(3))

list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

In [65]:
""" 새로운 방식 """
def chain(*iterables):
    for i in iterables:
        yield from i  # 16장에서 자세히 다룬다.

list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

### 14.11 반복형을 리듀스하는 함수
반복형을 입력받아 하나의 값을 반환한다. 흔히 '리듀스', '폴딩', 혹은 '누적' 함수라고 한다.

In [66]:
print(all([1, 2, 3]))
print(all([1, 0, 3]))
print(all([]))

True
False
True


In [67]:
print(any([1, 2, 3]))
print(any([1, 0, 3]))
print(any([0, 0.0]))
print(any([]))

True
True
False
False


In [68]:
g = (n for n in [0, 0.0, 7, 8])
print(any(g))  # 단락 평가를 하는 경우에는 결과가 확정되는 즉시 반복자 소비를 중단한다.
print(next(g)) # 7을 만나는 순간 True가 확정적이므로 중단된 7의 다음 인덱스 데이터인 8이 출력된다.

True
8


In [69]:
### 14.12 iter() 함수 들여다보기


In [70]:
""" 1이 나올 때까지 육면체 주사위를 굴리기 위해 iter() 함수를 사용하는 방법 """
from random import *

def d6():
    return randint(1, 6)

d6_iter = iter(d6, 1) # 1 is end of a sequence
d6_iter

#The syntax of iter() method is:

#iter(object[, sentinel])
#Recommended Reading: Python iterators and how to work with them.

#iter() Parameters
#The iter() method takes two parameters:

#object - object whose iterator has to be created (can be sets, tuples, etc.)
#sentinel (Optional) - special value that is used to represent the end of a sequence

In [71]:
for roll in d6_iter:
    print(roll)

2
3
4
3
3
4
5
4
3
4
5
5
6
4
4


다음 코드는 파일에서 빈 줄을 발견하거나 파일의 끝에 도달할 때까지 한 줄씩 읽어서 처리한다.
```
with open('mydata.txt') as fp:
    for line in iter(pf.readline, ''):
        process_line(line)
```

### 14.13 사례연구 : 데이터베이스 변환 유틸리니 안의 제너레이터
code4lib 제널에 게재된 'ISIS에서 카우치DB로: 참고문헌 레코드에 대한 데이터베이스 및 데이터 모델(From ISIS to CouchDB and Data Models for Bibliographic Record'를 참고하라.
<li> 논문: http://journal.code4lib.org/articles/4893 </li>
<li> 소스코드: http://bit.ly/1HGqzzT </li>


### - 뒷 이야기 -
함수 본체 안에 묻혀있는 yield 문은 의미가 완전히 다르다고 충분히 경고하지 못한다. [예제 14-24]에서 f()를 호출하면 제너레이터가 생성되지 않고 무한 루프에 빠진다. yield 키워드는 자신을 내포한 함수만 제너레이터 함수로 만들기 때문이다. 제너레이터 함수가 일반 함수처럼 보이지만, 함수 호출을 통해 다른 제너레이터 함수에 위힘할 수 없다. 

새로 추가된 yield from 구문은 파이썬 제너레이터나 코루틴이 내부 for 루프를 우회할 필요 없이 작업을 다른 함수에 위임할 수 있게 해준다. 따라서 필자는 코루틴이 별도의 키워드로 정의되어야 한다고 생각한다. 

In [72]:
""" [예제 14-24] 생성 과정을 간단히 추상화하는 것처럼 보이는 함수 코드, 무한루트에 빠짐 """
def f():
    def do_yield(n):
        yield n
    x = 0
    while True:
        x += 1
        do_yield(x)

In [73]:
""" [예제 14-25] 생성 과정을 실제로 간단히 추상화한 코드 """
def f():
    def do_yield(n):
        yield n
    x = 0
    while True:
        x += 1
        yield from do_yield(x)

제너레이터는 파이썬 제너레이터 객체를 사용하지 않고 [예제 14-26] 처럼 만들 수 있다. 하지만 바보 같은 사례이다. 실제로 파이썬 프로그래머들은 제너레이터(yield를 사용한 함수)와 반복자를 엄격히 구분하지 않는다. [예제 14-27]은 파이썬스럽게 구현한 것이다.

In [74]:
""" [예제 14-26] GeneratorType 객체를 사용하지 않고 구현한 피보나치 수열 제너레이터 """

class Fibonacci:
    def __iter__(self):
        return FibonacciGenerator()
    
class FibonacciGenerator:
    def __init__(self):
        self.a = 0
        self.b = 1    
        
    def __next__(self):
        result = self.a
        self.a, self.b = self.b, self.a + self.b
        return result
    
    def __iter__(self):
        return self

In [75]:
""" [예제 14-27] 파이서닉 코드 """
def fibonacci():
    a, b = 0, 1
    while True:
        yield a 
        a, b = b, a+b